## Imports and Seed

In [1]:
import tensorflow as tf
import numpy as np
import io
import os
import random
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from PIL import Image
import scipy.io as sio
import joblib

from plotly.subplots import make_subplots
import plotly.graph_objects as go

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

2.7.0


In [2]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

## Load Data and Preprocess

In [3]:
dataset = pd.read_csv('Training.csv')

# split into test and training datasets
test_size = int(dataset.shape[0]//10) # size of the test set is 10% of the training

X_train_raw = dataset.iloc[:-test_size]
X_test_raw = dataset.iloc[-test_size:]

#normalize the data

# choice of scaler
scaler = MinMaxScaler()

# fitting the scaler
scaler.fit(X_train_raw)

#transform X_train_raw and X_test_raw
X_pp_train = scaler.transform(X_train_raw)
X_pp_test = scaler.transform(X_test_raw)

# scaler returns numpy arrays (type = ndarray). Reconvert to dataframe for better plotting
data = np.concatenate((X_pp_train,X_pp_test))
data = pd.DataFrame(data, columns = dataset.columns)
X_pp_train = data.iloc[:-test_size]
X_pp_test = data.iloc[-test_size:]

# save the scikit learn scaler 
scaler_dir = 'vars\\scalers'
if not os.path.exists(scaler_dir):
    os.makedirs(scaler_dir)

scaler_path = os.path.join(scaler_dir, 'min_max_scaler.gz')
joblib.dump(scaler, scaler_path)

['vars\\scalers\\min_max_scaler.gz']

## Create Sequences

In [4]:
# First objective should be to predict the next quarter of test length. You can look at the instructions for the competition for 
# more information. So the total length to predict is c_telescope = 1152. In the first part of the competition we must predict 3 
# quarters of the total test sequences, i.e. 3/4 * 1152 = 864. One single quarter is made up of 1152/4 = 288. I believe, whatever
# the final structure of the model, that we could firs predict a quarter and then recgressively predict the rest.

quarter_tel = (1152//4) #288

# We can try to predict the quarter in 4 steps of 36

reg = 4
telescope = 80
assert quarter_tel%4 == 0

# target labels are all the labels
target_labels = dataset.columns

# Hyper parameters for how to build the sequences. The total length of the dataset is 68528, so quite large. I would take the
# usual length to start with. We will obtain a very large number of sequences. Note that in 
# the inspection of the data the beginning of the sequence seemed quite different from the later evolution. One could consider removing 
# the first part as a possibility. 

window = 300
stride = 10

In [5]:
# build sequence function
def build_sequences(df, target_labels=['pollution'], window=200, stride = 20, telescope=100):
    # check to avoid errrors
    assert window % stride == 0
    dataset = []
    labels = []
    temp_df = df.copy().values #copy() makes a deep copy
    temp_labels = df[target_labels].copy().values
    need_4_padding = len(df)%window != 0
    
    if(need_4_padding):
        padding_len = window-len(df)%window
        padding = np.zeros((padding_len, temp_df.shape[1]), dtype='float64')
        temp_df = np.concatenate((padding, df))
        padding = np.zeros((padding_len, temp_labels.shape[1]), dtype='float64')
        temp_labels = np.concatenate((padding, temp_labels))
        assert len(temp_df)%window == 0
        
    for idx in np.arange(0, len(temp_df)-window-telescope, stride):
        dataset.append(temp_df[idx:idx+window])
        labels.append(temp_labels[idx+window:idx+window+telescope])
    dataset = np.array(dataset)
    labels = np.array(labels)
    return dataset, labels

In [6]:
X_train, y_train = build_sequences(
    X_pp_train,
    target_labels=target_labels,
    window=window,
    stride=stride,
    telescope=telescope
    )
X_test, y_test = build_sequences(
    X_pp_test,
    target_labels=target_labels,
    window=window,
    stride=stride,
    telescope=telescope
    )

## Tokenize the TimeSeries

In [7]:
# "tokenize" the sequences
token_length = 20

def tokenize(X, token_length=20, step = 20):
    assert window%token_length == 0
    return X.reshape(X.shape[0], -1, token_length, X_train.shape[2])


# Xtokens = tokenize(X_train, token_length)
# ytokens = tokenize(y_train, token_length)
# print(Xtokens.shape)
# print(ytokens.shape)

In [10]:
# Inspect tokenization

# plotting function 
def inspect_tokenization(X, tokens, columns, idx=None):
    if(idx==None):
        idx=np.random.randint(0,len(X))
    figs, axs = plt.subplots(len(columns), 1, sharex=True, figsize=(17,17))
    for i, col in enumerate(columns):
        for j in range(tokens.shape[1]):
            _token = tokens[idx, j, ...]
            axs[i].plot(np.arange(j*token_length, (j+1)*token_length), _token[:, i], color='orange')
        axs[i].plot(np.arange(len(X[0,:,i])), X[idx,:,i])
        axs[i].set_title(col)
        axs[i].set_ylim(0,1)
        # print(np.arange(len(X[0,:,i]), len(X_train[0,:,i])+1))
    plt.show()    

## Embed and Deembed Functions

In [11]:
models_dir = 'models'
autoencoder = tfk.models.load_model(os.path.join(models_dir, 'autoencoder_w20B'))

encoder = autoencoder.get_layer('Encoder')
encoder.summary()

decoder_input = tfkl.Input(shape=(32), name='decoder_input')
repeatlayer = autoencoder.get_layer('latent_representation')(decoder_input)
d = autoencoder.get_layer('decoder')
decoder = tfk.Model(inputs=decoder_input, outputs=d(repeatlayer), name='Decoder')
decoder.summary()

Model: "Encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_6 (Bidirectio  (None, 20, 128)          36864     
 nal)                                                            
                                                                 
 bidirectional_7 (Bidirectio  (None, 20, 64)           41216     
 nal)                                                            
                                                                 
 bidirectional_8 (Bidirectio  (None, 32)               10368     
 nal)                                                            
                                                                 
Total params: 88,448
Trainable params: 88,448
Non-trainable params: 0
_________________________________________________________________
Model: "Decoder"
_________________________________________________________________
 Layer (type)                Output Sh

In [12]:
# simple embed and deembed functions

def embed(tokens):
    t = tokens.reshape(-1, tokens.shape[-2], tokens.shape[-1])
    embedded = encoder.predict(t)
    embedded = embedded.reshape(tokens.shape[0], tokens.shape[1], -1)
    return embedded
        
def deembed(e):
    t = e.reshape(-1, e.shape[-1])
    d = decoder.predict(t)
    d = d.reshape(e.shape[0], e.shape[1], token_length, 7)
    return d        
    

In [13]:
# verify the embedding and inspect the embedding/deembedding functions
# inspect_tokenization(X_train, deembed(embed(tokenize(X_train))), dataset.columns)

### Embed the Data

In [14]:
X_emb = embed(tokenize(X_train))
y_emb = embed(tokenize(y_train))

X_emb.shape, y_emb.shape

((6142, 15, 32), (6142, 4, 32))

## Build the Seq2Seq Model

In [15]:
# hyperparams

# model hyperparams
input_shape = X_emb.shape[1:]
output_shape = y_emb.shape[1:]

batch_size = 512
epochs = 200

In [16]:
# Function to create the model 
# Train with teacher forcing

def sequence2sequence(
    embedded_dims, input_shape, telescope
):
    # Build Encoder-Decoder Model
    # ---------------------------
    
    # ENCODER
    # -------
    encoder_input = tf.keras.Input(
        shape=input_shape, name = 'encoder_input'
    )
    lstm1 = tfkl.LSTM(128, activation='relu', return_sequences=True, name='lstm1_encoder')(encoder_input)
    encoder_output, h, c = tfkl.LSTM(128, activation='relu', return_state=True, name='lstm2_encoder')(lstm1)    
    encoder_state = [h, c] 
    
    # note that both h and c are passed to the decoder to initialize the state
    # this is why we needed return_state = True
    
    #DECODER
    #-------
    decoder_input = tf.keras.Input(shape = output_shape, name='decoder_input')
    # initialize the decoder state with the final encoder state
    decoder_lstm, _, _ = tfkl.LSTM(units = 128, return_sequences = True, return_state = True, name ='lstm_dec')(decoder_input, initial_state = encoder_state)
    decoder_out = tf.keras.layers.Dense(
        embedded_dims, 
        activation='relu',
        name = 'decoder_out'
    )(decoder_lstm)
    
    #MODEL 
    model = tf.keras.Model([encoder_input, decoder_input], decoder_out)
    
    #---------------
    # compile training model
    model.compile(
        loss=tfk.losses.MeanSquaredError(), 
        # so we don't have to use 1-hot encoding
        optimizer = tfk.optimizers.Adam(learning_rate = 1e-2),
        # large learning rate, probably because the net is simple and 
        # it takes a long time to train 1 epoch
        metrics = ['mae']
    )
    
    return model, encoder_state

In [17]:
seq2seq_model, encoder_state = sequence2sequence(
    embedded_dims=32, input_shape=input_shape, telescope = 4
)

seq2seq_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input (InputLayer)     [(None, 15, 32)]     0           []                               
                                                                                                  
 lstm1_encoder (LSTM)           (None, 15, 128)      82432       ['encoder_input[0][0]']          
                                                                                                  
 decoder_input (InputLayer)     [(None, 4, 32)]      0           []                               
                                                                                                  
 lstm2_encoder (LSTM)           [(None, 128),        131584      ['lstm1_encoder[0][0]']          
                                 (None, 128),                                                 

In [18]:
# embed the first input is set to zero
encoder_input = X_emb
padding = np.zeros([y_emb.shape[0], 1, y_emb.shape[2]])
decoder_input = np.append( padding, y_emb,axis=1 )[:, :-1, :]
target = y_emb

In [19]:
decoder_input.shape, target.shape


((6142, 4, 32), (6142, 4, 32))

In [20]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

#train the RNN
history = seq2seq_model.fit(
    x=[encoder_input, decoder_input],
    y=target,
    batch_size = batch_size,
    epochs=1000,
    validation_split=.1,
    callbacks=[
        tfk.callbacks.EarlyStopping(
            monitor='val_loss',
            mode='min',
            patience=10,
            restore_best_weights=True
            ),
        tfk.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            mode='min',
            patience=5,
            factor=0.5,
            min_lr=1e-2
            ),
        ]
    ).history

Epoch 1/1000
11/11 [==============================] - 3s 117ms/step - loss: 0.0312 - mae: 0.1396 - val_loss: 0.0256 - val_mae: 0.1208 - lr: 0.0100
Epoch 2/1000
11/11 [==============================] - 1s 77ms/step - loss: 0.0242 - mae: 0.1133 - val_loss: 0.0238 - val_mae: 0.1092 - lr: 0.0100
Epoch 3/1000
11/11 [==============================] - 1s 76ms/step - loss: 0.0235 - mae: 0.1075 - val_loss: 0.0236 - val_mae: 0.1066 - lr: 0.0100
Epoch 4/1000
11/11 [==============================] - 1s 76ms/step - loss: 0.0234 - mae: 0.1057 - val_loss: 0.0236 - val_mae: 0.1054 - lr: 0.0100
Epoch 5/1000
11/11 [==============================] - 1s 77ms/step - loss: 0.0233 - mae: 0.1051 - val_loss: 0.0236 - val_mae: 0.1052 - lr: 0.0100
Epoch 6/1000
11/11 [==============================] - 1s 76ms/step - loss: 0.0233 - mae: 0.1047 - val_loss: 0.0235 - val_mae: 0.1049 - lr: 0.0100
Epoch 7/1000
11/11 [==============================] - 1s 78ms/step - loss: 0.0233 - mae: 0.1045 - val_loss: 0.0235 - val_ma

Epoch 57/1000
11/11 [==============================] - 1s 75ms/step - loss: 0.0232 - mae: 0.1023 - val_loss: 0.0234 - val_mae: 0.1029 - lr: 0.0100
Epoch 58/1000
11/11 [==============================] - 1s 81ms/step - loss: 0.0232 - mae: 0.1023 - val_loss: 0.0234 - val_mae: 0.1028 - lr: 0.0100
Epoch 59/1000
11/11 [==============================] - 1s 76ms/step - loss: 0.0232 - mae: 0.1023 - val_loss: 0.0234 - val_mae: 0.1028 - lr: 0.0100
Epoch 60/1000
11/11 [==============================] - 1s 75ms/step - loss: 0.0231 - mae: 0.1021 - val_loss: 0.0238 - val_mae: 0.1079 - lr: 0.0100
Epoch 61/1000
11/11 [==============================] - 1s 75ms/step - loss: 0.0225 - mae: 0.1030 - val_loss: 0.0225 - val_mae: 0.0998 - lr: 0.0100
Epoch 62/1000
11/11 [==============================] - 1s 75ms/step - loss: 0.0222 - mae: 0.0991 - val_loss: 0.0224 - val_mae: 0.0984 - lr: 0.0100
Epoch 63/1000
11/11 [==============================] - 1s 76ms/step - loss: 0.0221 - mae: 0.0980 - val_loss: 0.0224 - 

Epoch 113/1000
11/11 [==============================] - 1s 77ms/step - loss: 0.0220 - mae: 0.0964 - val_loss: 0.0223 - val_mae: 0.0970 - lr: 0.0100
Epoch 114/1000
11/11 [==============================] - 1s 76ms/step - loss: 0.0220 - mae: 0.0964 - val_loss: 0.0223 - val_mae: 0.0971 - lr: 0.0100
Epoch 115/1000
11/11 [==============================] - 1s 76ms/step - loss: 0.0220 - mae: 0.0964 - val_loss: 0.0223 - val_mae: 0.0970 - lr: 0.0100
Epoch 116/1000
11/11 [==============================] - 1s 75ms/step - loss: 0.0220 - mae: 0.0964 - val_loss: 0.0223 - val_mae: 0.0971 - lr: 0.0100
Epoch 117/1000
11/11 [==============================] - 1s 76ms/step - loss: 0.0220 - mae: 0.0964 - val_loss: 0.0224 - val_mae: 0.0972 - lr: 0.0100
Epoch 118/1000
11/11 [==============================] - 1s 76ms/step - loss: 0.0220 - mae: 0.0964 - val_loss: 0.0223 - val_mae: 0.0970 - lr: 0.0100
Epoch 119/1000
11/11 [==============================] - 1s 79ms/step - loss: 0.0220 - mae: 0.0963 - val_loss: 0.

11/11 [==============================] - 1s 78ms/step - loss: 0.0220 - mae: 0.0962 - val_loss: 0.0223 - val_mae: 0.0969 - lr: 0.0100
Epoch 169/1000
11/11 [==============================] - 1s 80ms/step - loss: 0.0220 - mae: 0.0961 - val_loss: 0.0223 - val_mae: 0.0969 - lr: 0.0100
Epoch 170/1000
11/11 [==============================] - 1s 77ms/step - loss: 0.0220 - mae: 0.0961 - val_loss: 0.0223 - val_mae: 0.0969 - lr: 0.0100
Epoch 171/1000
11/11 [==============================] - 1s 76ms/step - loss: 0.0220 - mae: 0.0961 - val_loss: 0.0223 - val_mae: 0.0969 - lr: 0.0100
Epoch 172/1000
11/11 [==============================] - 1s 76ms/step - loss: 0.0220 - mae: 0.0961 - val_loss: 0.0223 - val_mae: 0.0968 - lr: 0.0100
Epoch 173/1000
11/11 [==============================] - 1s 78ms/step - loss: 0.0220 - mae: 0.0961 - val_loss: 0.0223 - val_mae: 0.0969 - lr: 0.0100
Epoch 174/1000
11/11 [==============================] - 1s 76ms/step - loss: 0.0220 - mae: 0.0962 - val_loss: 0.0223 - val_mae:

In [21]:
InteractiveSession.close(self=session)